### Preliminaries
If you want to normalize a vector to L1-norm or L2-norm, use:

In [1]:
from __future__ import print_function, division
import numpy as np

pr = np.array([1,2,3])
print("L1-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,1)))
print("L2-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,2)))

L1-norm of [1 2 3] is [0.16666667 0.33333333 0.5       ]
L2-norm of [1 2 3] is [0.26726124 0.53452248 0.80178373]


# Exercise 3: Link based ranking
## Question 1 - Page Rank (Eigen-vector method)
Consider a tiny Web with three pages A, B and C with no inlinks,
and with initial PageRank = 1. Initially, none of the pages link to
any other pages and none link to them. 
Answer the following questions, and calculate the PageRank for
each question.

1. Link page A to page B.
2. Link all pages to each other.
3. Link page A to both B and C, and link pages B and C to A.
4. Use the previous links and add a link from page C to page B.

Hints: 
+ We are using the theoretical PageRank computation (without source of rank). See slide "Transition Matrix for Random Walker" in the lecture note. **Columns of link matrix are from-vertex, rows of link matrix are to-vertex**. We take the eigenvector with the largest eigenvalue.
+ We only care about final ranking of the probability vector. You can choose the normalization (or not) of your choice).

In [91]:
# Implement your code here
def pagerank_eigen(L):
#   Construct transition probability matrix from L
    R = np.zeros(np.shape(L))
    
    for row in range(0, np.shape(L)[0]):
        for col in range(0, np.shape(L)[1]):
            R[row,col] = L[row,col] / np.sum(L[:,col])
        
#  Compute eigen-vectors and eigen-values of R
#   eig vs svd ?
    U,S,Vt = np.linalg.svd(R, full_matrices=True)
    eigenvalues, eigenvectors = np.linalg.eig(R)
        
#   Take the eigen-vector with maximum eigven-value
    p = eigenvectors[:,0]
    return (R,p)

In [93]:
# Test with the question, e.g.
L = np.matrix([
    [0,0,1], 
    [1,0,0], 
    [1,1,0]
])
R,p = pagerank_eigen(L)

# need to normalize
print("L={0}\nR={1}\np={2}".format(L,R,p))

L=[[0 0 1]
 [1 0 0]
 [1 1 0]]
R=[[0.  0.  1. ]
 [0.5 0.  0. ]
 [0.5 1.  0. ]]
p=[-0.66666667+0.j -0.33333333+0.j -0.66666667+0.j]


## Question 2 - Page Rank (Iterative method)

The eigen-vector method has some numerical issues (when computing eigen-vector) and not scalable with large datasets.

We will apply the iterative method in the slide "Practical Computation of PageRank" of the lecture.

Dataset for practice: https://snap.stanford.edu/data/ca-GrQc.html. It is available within the same folder of this github.

In [86]:
def pagerank_iterative(L):
    
    #   Construct transition probability matrix from L
    R = np.zeros(np.shape(L))    
    for row in range(0, np.shape(L)[0]):
        for col in range(0, np.shape(L)[1]):
            R[row,col] = L[row,col] / np.sum(L[:,col])
    
    N = L.shape[0]
    e = np.ones(shape=(N,1))
    
    q = 0.9          # teleport probability
    p = e            # arbitrary start vector
    delta = 1        # error of current estimate
    epsilon = 0.001  # error threshold    
    
    i = 0
    while delta > epsilon:
        p_prev = p
        p = np.matmul(q*R,p)
        p = p + (1-q)/N*e
        delta = np.linalg.norm(p - p_prev,2)
        i += 1

    print("Converged after {0} iterations. Ranking vector: p={1}".format(i, p[:,0]))
    return R,p

#### Test with the dataset


In [87]:
# Construct link matrix from file
f = open("ca-GrQc.txt")
L = np.matrix([
    [0,0,1], 
    [1,0,0], 
    [1,1,0]
])

In [88]:
# Run PageRank
R, p = pagerank_iterative(L)
print("Ranking vector: p={0}".format(p[:,0]))

Converged after 47 iterations. Ranking vector: p=[0.39755738 0.21251694 0.40406498]
Ranking vector: p=[0.39755738 0.21251694 0.40406498]


## Question 3 - Ranking Methodology (Hard)

1. Give a directed graph, as small as possible, satisfying all the properties mentioned below:

    1. There exists a path from node i to node j for all nodes i,j in the directed
graph. Recall, with this property the jump to an arbitrary node in PageRank
is not required, so that you can set q = 1 (refer lecture slides).

    2. HITS authority ranking and PageRank ranking of the graph nodes are different.

2. Give intuition/methodology on how you constructed such a directed graph with
the properties described in (a).

3. Are there specific graph structures with arbitrarily large instances where PageRank
ranking and HITS authority ranking are the same?

## Question 4 - Hub and Authority

### a)

Let the adjacency matrix for a graph of four vertices ($n_1$ to $n_4$) be
as follows:

$
A =
  \begin{bmatrix}
	0 & 1 & 1 & 1  \\
	0 & 0 & 1 & 1 \\
	1 & 0 & 0 & 1 \\
	0 & 0 & 0 & 1 \\
  \end{bmatrix}
$

Calculate the authority and hub scores for this graph using the
HITS algorithm with k = 6, and identify the best authority and
hub nodes.

### b)
Apply the HITS algorithm to the dataset: https://snap.stanford.edu/data/ca-GrQc.html

**Hint:** We follow the slide "HITS algorithm" in the lecture. **Denote $x$ as authority vector and $y$ as hub vector**. You can use matrix multiplication for the update steps in the slide "Convergence of HITS". Note that rows of adjacency matrix is from-vertex and columns of adjacency matrix is to-vertex.

In [90]:
# You can implement your code following this template.
def hits_iterative(A, k=10):
    N = A.shape[0]

    x0 = 1 / (N*N) * np.ones(N) # authority vector
    y0 = 1 / (N*N) * np.ones(N) # hub vector
    xprev, yprev = x0, y0
    
    # authority sum row
    # hub sum column
    
    # For advanced exercise: define a convergence condition instead of k iterations
    for l in range(0,k):
        
        for i in range(N):
            x[i] = xprev[i] * A[:,i]
            y[i] = yprev[i] * A[i,:]
            xprev = x / np.linalg.norm(x,2)
            yprev = y / np.linalg.norm(y,2)
        
    return xprev, yprev